In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import pandas as pd
import numpy as np
# code for rdkit taken from:
# https://xinhaoli74.github.io/posts/2020/04/RDKit-Cheatsheet/

In [2]:
# STEP 0
# First read files
# we are reading in the desired molecule from the .smi file in the directory and then converting it to a .gro for GROMACS
# you are free to start with any .gro file you have yourself and skip this cell and the next cell
file_name = 'chembl_nomore15_CHNO_diverse10K.smi'
with open(f'{file_name}', "r") as ins:
    smiles = []
    for line in ins:
        smiles.append(line.split('\n')[0])
smiles = smiles[1:]
print('# of SMILES:', len(smiles))

# of SMILES: 10000


In [4]:
# STEP 1
# Use rdkit to make coordinates from smiles
# Variables that might need to be changed:
# n_smiles - depends on how many molecules we want
# out_path - path to which gros should be added
# box_scale - factor by which to multiply the largest distance
#             the final box will be of size largest_distance * box_scale

n_smiles = 2
smiles = smiles[0:n_smiles]
box_scale = 3
for midx in range(len(smiles)):
    mol = smiles[midx]
    m = Chem.MolFromSmiles(mol)
    m_Hs = Chem.AddHs(m)
    n_atoms = m_Hs.GetNumAtoms()
    AllChem.EmbedMolecule(m_Hs,randomSeed=0xf00d)
    coords3d = Chem.MolToMolBlock(m_Hs).split('\n')[4:4+n_atoms]
    out_path = os.getcwd()
    out = open(f'{out_path}/mol{midx}.gro', 'w+')
    out.write(f'\n{n_atoms}\n')
    xs = []
    ys = []
    zs = []
    for idx in range(n_atoms):
        ln = coords3d[idx]
        atom_type = ln.split()[3]
        coords = [float(i)/10 for i in ln.split()[0:3]]
        xs.append(coords[0])
        ys.append(coords[1])
        zs.append(coords[2])
        coords = ' '.join(['{:7.3f}'.format(i) for i in coords])
        atom_idx = '{:0>2}'.format(idx)
        fidx = '{:2}'.format(idx)
        out.write(f'    1UNK    {atom_type}{atom_idx}   {fidx} {coords}\n')
    x_diff = max(xs) - min(xs)
    y_diff = max(ys) - min(ys)
    z_diff = max(zs) - min(zs)
    box = max([x_diff, y_diff, z_diff])*box_scale
    box = '{:.5f}'.format(box)
    out.write(f'   {box}   {box}   {box}')
    out.close()

In [ ]:
# STEP 2
# Because I am lazy I did not write a script to add water molecules
# instead I wrote a bash script that uses gromacs
# copy the code below into a file called 'add_waters.sh'
# then save it and run it from the terminal with ./add_waters.sh or bash add_waters.sh
# for each molecule the code will create a gro with up to 10 added water molecules
# I am not sure how many it will add
# you can use another number by changing -nmol 10 to -nmol new_num 
# where new_num is the number your new number
# MAKE SURE water.gro EXISTS IN SAME FOLDER

# in this case, we solvate with roughly 10 waters, but again you can skip this step and solvate with whatever you want
# just make sure the solvated molecules have the correct SOL tag in the final .gro file

for i in mol*.gro
do
        fname="${i%.*}"
        gmx editconf -f $i -c -o ${fname}_centered.gro
done


for i in mol*centered.gro
do
        fname="${i%.*}"
        gmx insert-molecules -f $i -ci water.gro -nmol 10 -o ${fname}_solv.gro
done

In [7]:
# STEP 3
# Create input files for QChem MD run and create xyz files
# Variables that might need to be changed:
# path - should match out_path above, ie path to which gros were added
# qchem_keyes - keywords for qchem MD run
# qchem_path - path to which qchem inpus will be added

# you are also free to use any MD program to sample structures from
# we use ab initio MD via QCHEM here, but you can also sample structures from classical mechanics in gromacs
# if you choose to do that you have to create your own topologies and all that, so keep that in mind
# also using ab initio makes it so our MD simulations cant be too long

path = os.getcwd()
gros = [fl for fl in os.listdir(f'{path}') if fl.endswith('solv.gro')]

qchem_keys = """
$rem
   JOBTYPE              aimd
   AIMD_METHOD          bomd
   METHOD               wb97
   BASIS                6-31g*
   TIME_STEP            20        (20 a.u. = 0.48 fs) 
   AIMD_STEPS           100
   AIMD_INIT_VELOC      thermal
   AIMD_TEMP            298
   FOCK_EXTRAP_ORDER    6         request Fock matrix extrapolation
   FOCK_EXTRAP_POINTS   12 
   SCF_CONVERGENCE      6
   MAX_SCF_CYCLES       100
$end"""

diverse_dict = {'name' : [], 'qm_atoms' : [], 'n_waters' : []}
for gro in gros:
    name = gro.replace('.gro', '')
    qm_atoms = 0
    n_waters = 0
    qchem_path = f'{path}'
    out_qchem = open(f'{qchem_path}/{name}.in', 'w+')
    out_qchem.write('$molecule\n   0  1\n')
    with open(f'{path}/{gro}') as fl:
        gro_lns = fl.readlines()[2:-1]
    for ln in gro_lns:
        atom_type = ln.split()[1][0]
        coords = [float(i)*10 for i in ln.split()[3:]]
        coords = ' '.join(['{:10.5f}'.format(i) for i in coords])
        out_qchem.write(f'   {atom_type} {coords}\n')
        if '1UNK' in ln:
            qm_atoms += 1
        elif 'SOL' in ln and ln.split()[1] == 'OW':
            n_waters += 1
    out_qchem.write(qchem_keys)
    out_qchem.close()
    diverse_dict['name'].append(name)
    diverse_dict['qm_atoms'].append(qm_atoms)
    diverse_dict['n_waters'].append(n_waters)

In [8]:
# Save a df with name of system, n_qm_atoms, n_water and total_atoms
diverse_mols_df = pd.DataFrame(diverse_dict)
diverse_mols_df['total_atoms'] = diverse_mols_df['qm_atoms'] + 3*(diverse_mols_df['n_waters'])
diverse_mols_df.to_csv(f'{path}/diverse_mols_systems.csv')

# STEP 4 is to run MD on cluster
# we generated the QCHEM .in files in an earlier step, so just run those through whatever installation of QCHEM you have
# or run whatever MD program you will be using, just make sure that your extracted snapshots are in a .gro file format

In [17]:
# STEP 5
# AFTER RUNNING MD
# Extract random structures from MD output file
# Variables that might need to be changed:
# path - path containing centered and solvated gros produced after running bash script
# qchem_path - path used above
# min_step - min step to extract
# max_step - max possible step to extract
# n_steps - num of random steps to extract between (min_step, max_step)

# here we take the qchem .out file and randomly grab structures from it, so you need the .out file in the directory after you ran the MD
# again most of this can be skipped if you choose to not use QCHEM/aimd, but you need to extract your snapshots in your own way
path = os.getcwd()
qchem_path = path
min_step = 50
max_step = 100
n_steps = 4
out_mds = [fl for fl in os.listdir(f'{qchem_path}') if fl.endswith('.out') and 'slurm' not in fl]
for out_md in out_mds:
    name = out_md.replace('.out', '')
    with open(f'{path}/{name}.gro') as fl:
        gro_lns = fl.readlines()[2:]
    with open(f'{path}/{out_md}', 'r') as fl:
        md_lns = fl.readlines()
    steps = np.random.choice(range(min_step, max_step), n_steps)
    total_atoms = len(gro_lns)-1
    for step in steps:
        for ln in md_lns:
            if f'TIME STEP #{step} ' in ln:
                c_in = md_lns.index(ln)+7
                c_fn = c_in + total_atoms
                md_coords = md_lns[c_in:c_fn]
                out_gro = open(f'{qchem_path}/{name}_{step}.gro', 'w+')
                out_gro.write(f'\n{total_atoms}\n')
                #for c in md_lns[c_in:c_fn]:
                for atom in range(total_atoms):
                    coords = [float(i)/10 for i in md_coords[atom].split()[2:]]
                    coords = ' '.join(['{:10.5f}'.format(i) for i in coords])
                    mol_id = gro_lns[atom].split()[0]
                    atom_type_id = gro_lns[atom].split()[1]
                    if atom_type_id == 'OW':
                        atom_num = '{:3}'.format(int(gro_lns[atom].split()[2]))
                    else:
                        atom_num = '{:2}'.format(int(gro_lns[atom].split()[2]))
                    out_gro.write(f'    {mol_id}    {atom_type_id}   {atom_num} {coords}\n')
                out_gro.write(gro_lns[-1])
                out_gro.close()